![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.svg)
# Pipeline Notebook - AutoAI Notebook v1.22.4

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimal results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for retrieving data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.10 and scikit-learn 1.1.1.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data with train and test data split](#read)<br>
&nbsp;&nbsp;[Create pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
&nbsp;&nbsp;[Forecast](#fore)<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm-watsonx-ai,
 - autoai-ts-libs,
 - scikit-learn


In [1]:
!pip install ibm-watsonx-ai | tail -n 1
!pip install autoai-ts-libs==3.0.23 | tail -n 1
!pip install -U scikit-learn==1.1.1 | tail -n 1

^C
ERROR: Operation cancelled by user


<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data with train and test data split

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`

In [2]:
import pandas
dataset = pandas.read_csv("../datasets/TSLA.csv")
dataset.drop_duplicates()
dataset.sample()

,Date,Open,High,Low,Close,Adj Close,Volume
236,2017-07-25,69.0,69.120003,66.830002,67.919998,67.919998,34946000


In [3]:
df = dataset.dropna(subset=['Adj Close', 'Volume'], inplace=False)
df = df.sort_values(by='Date')
df = df.drop('Date', axis=1)
X = df
y = df[['Adj Close', 'Volume']]

In [4]:
test_size = 0.1

# Obliczenie indeksu, na którym należy dokonać podziału
split_idx = int(len(X) * (1 - test_size))

# Podział danych na treningowe i testowe
train_X = X[:split_idx]
test_X = X[split_idx:]
train_y = y[:split_idx]
test_y = y[split_idx:]

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [5]:
from autoai_ts_libs.utils.ts_pipeline import TSPipeline
from autoai_ts_libs.transforms.imputers import linear
from autoai_ts_libs.watfore.watfore_forecasters import WatForeForecaster

#### Pipeline definition.

In [6]:
linear = linear(missing_val_identifier=float("nan"))
wat_fore_forecaster = WatForeForecaster(
    algorithm="bats",
    diff_eta=True,
    difference_all_data=True,
    disable_difference=False,
    feature_column_indices=[0, 1],
    log_transformed=False,
    lookback_win=33,
    prediction_horizon=6,
    target_column_indices=[0, 1],
)
pipeline = TSPipeline(
    steps=[
        ("linear_imputer", linear),
        (
            "<class 'ai4ml_ts.estimators.watfore_forecasters.WatForeForecaster'>",
            wat_fore_forecaster,
        ),
    ],
    feature_columns=[0, 1],
    prediction_horizon=6,
    target_columns=[0, 1],
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [7]:
# from autoai_ts_libs.utils.metrics import get_scorer

# scorer = get_scorer("neg_avg_symmetric_mean_absolute_percentage_error")

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [8]:
pipeline.fit(train_X.values, train_y.values);

Cannot find seasons
Cannot find seasons


<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [9]:
# score = scorer(pipeline, test_X.values, test_y.values)
# print(score)

In [ ]:
pipeline.predict(test_X.values)

<a id="forecast"></a>
## Forecast

Forecast the future values of targets.

In [10]:
pipeline.predict()

array([[802.0837964 , 819.00082101],
       [803.7072975 , 820.65821711],
       [805.33079859, 822.31561321],
       [806.95429969, 823.9730093 ],
       [808.57780079, 825.6304054 ],
       [810.20130189, 827.2878015 ]])

### Forecasting with the new observations
Provide new observations values
For example:
 - `a,b` are both targets and features, taken from `experiment_metadata["prediction_columns"]`

```
import numpy as np

a_newObs = [12, 22, 35, 46]
b_newObs = [34, 23, 45, 34]
newObs = np.mat([a_newObs, b_newObs]).T
pipeline.predict(newObs)
```


<a id="summary_and_next_steps"></a>
# Summary and next steps
You successfully completed this notebook!
You learned how to use AutoAI pipeline definition to train the model.
Check out our [Online Documentation](https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/cpd/overview/relnotes-4.0.html) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___